# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [5]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

### GPT2: 30%

In [349]:
import collections
import re
import requests
import time
import json
from urllib.parse import parse_qs

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = self.full_dictionary

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:
            requests.get(link)
            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def build_dictionary(self, dictionary_file_location):
        with open(dictionary_file_location, "r") as text_file:
            full_dictionary = text_file.read().splitlines()
        return full_dictionary

    def guess(self, word):  # word input example: "_ p p _ e "
        clean_word = word[::2].replace("_", ".")
        
        len_word = len(clean_word)
        
        # Filter current dictionary based on the pattern
        self.current_dictionary = [w for w in self.current_dictionary if len(w) == len_word and re.match(clean_word, w)]
        
        # Count occurrences of all characters in the filtered dictionary
        full_dict_string = "".join(self.current_dictionary)
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()
        
        # Prioritize vowels over consonants
        vowels = "aeiou"
        letter_scores = {}
        
        for letter, instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                if letter in vowels:
                    letter_scores[letter] = instance_count * 2  # Weight vowels more
                else:
                    letter_scores[letter] = instance_count
        
        # Return the letter with the highest score
        if letter_scores:
            return max(letter_scores, key=letter_scores.get)
        
        # if no letter found, fallback to the full dictionary common letters
        sorted_letter_count = self.full_dictionary_common_letter_sorted
        for letter, instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                return letter
        
        return None

    def start_game(self, practice=True, verbose=True):
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice": practice})
        if response.get('status') == "approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print(f"Successfully started a new game! Game ID: {game_id}. # of tries remaining: {tries_remains}. Word: {word}.")
            while tries_remains > 0:
                guess_letter = self.guess(word)
                
                if guess_letter:
                    self.guessed_letters.append(guess_letter)
                    if verbose:
                        print(f"Guessing letter: {guess_letter}")
                    
                    try:
                        res = self.request("/guess_letter", {"request": "guess_letter", "game_id": game_id, "letter": guess_letter})
                    except HangmanAPIError:
                        print('HangmanAPIError exception caught on request.')
                        continue
                    except Exception as e:
                        print('Other exception caught on request.')
                        raise e
                    
                    if verbose:
                        print(f"Server response: {res}")
                    status = res.get('status')
                    tries_remains = res.get('tries_remains')
                    if status == "success":
                        if verbose:
                            print(f"Successfully finished game: {game_id}")
                        return True
                    elif status == "failed":
                        reason = res.get('reason', '# of tries exceeded!')
                        if verbose:
                            print(f"Failed game: {game_id}. Because of: {reason}")
                        return False
                    elif status == "ongoing":
                        word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status == "success"
    
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        if self.access_token:
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

### raks

In [342]:
import requests
import time
import collections
import re

class HangmanGame:
    def __init__(self, dictionary_file_location="words.txt"):
        self.full_dictionary = self.build_dictionary(dictionary_file_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.n_word_dictionary = self.build_n_word_dictionary(self.full_dictionary)
        self.popchars = "eiaou" if vowel else "eiano"
        self.popcharsratio = 0.55 if vowel else 0.625
        self.current_dictionary = self.full_dictionary
        self.guessed_letters = []

    def build_n_word_dictionary(self, word_list):
        n_word_dict = {i: [] for i in range(3, 30)}
        max_length = self.find_max_length(word_list)
        for count in range(3, max_length + 1):
            for word in word_list:
                if len(word) >= count:
                    for i in range(len(word) - count + 1):
                        n_word_dict[count].append(word[i:i + count])
        return n_word_dict

    @staticmethod
    def find_max_length(word_list):
        return max(len(word) for word in word_list)

    @staticmethod
    def build_dictionary(dictionary_file_location):
        with open(dictionary_file_location, "r") as text_file:
            return text_file.read().splitlines()

    def popchars_count(self, clean_word):
        count = sum(1 for char in clean_word if char in self.popchars)
        return count / len(clean_word) if len(clean_word) > 0 else 0.0

    @staticmethod
    def func(new_dictionary):
        dictx = collections.Counter()
        for words in new_dictionary:
            temp = collections.Counter(words)
            temp.update()
            dictx += temp
        return dictx

    def func2(self, clean_word):
        l = len(clean_word)
        return [dict_word for dict_word in self.n_word_dictionary[l] if re.match(clean_word, dict_word)]

    def guess(self, word):
#         clean_word = word.replace("_", ".")
        clean_word = word[::2].replace("_", ".")
        len_word = len(clean_word)
        new_dictionary = [dict_word for dict_word in self.current_dictionary if len(dict_word) == len_word and re.match(clean_word, dict_word)]
        self.current_dictionary = new_dictionary

        c = self.func(new_dictionary)
        sorted_letter_count = c.most_common()

        guess_letter = '!'
        for letter, _ in sorted_letter_count:
            if letter not in self.guessed_letters:
                if letter in self.popchars and self.popchars_count(clean_word) > self.popcharsratio:
                    self.guessed_letters.append(letter)
                    continue
                guess_letter = letter
                break

        if guess_letter == '!':
            new_dictionary = self.func2(clean_word)
            c = self.func(new_dictionary)
            sorted_letter_count = c.most_common()
            for letter, _ in sorted_letter_count:
                if letter not in self.guessed_letters:
                    if letter in self.popchars and self.popchars_count(clean_word) > self.popcharsratio:
                        self.guessed_letters.append(letter)
                        continue
                    guess_letter = letter
                    break

        if guess_letter == '!':
            for length_fraction in [2, 3]:
                x = int(len(clean_word) / length_fraction)
                if x >= 3:
                    c = collections.Counter()
                    for i in range(len(clean_word) - x + 1):
                        s = clean_word[i:i + x]
                        new_dictionary = self.func2(s)
                        temp = self.func(new_dictionary)
                        c += temp
                    sorted_letter_count = c.most_common()
                    for letter, _ in sorted_letter_count:
                        if letter not in self.guessed_letters:
                            guess_letter = letter
                            break
                if guess_letter != '!':
                    break

        if guess_letter == '!':
            for letter, _ in self.full_dictionary_common_letter_sorted:
                if letter not in self.guessed_letters:
                    if letter in self.popchars and self.popchars_count(clean_word) > self.popcharsratio:
                        self.guessed_letters.append(letter)
                        continue
                    guess_letter = letter
                    break
        return guess_letter

    def simulate_hangman(self, word):
        self.current_dictionary = self.full_dictionary
        self.guessed_letters = []

        max_lives = 6
        lives = max_lives
        guessed_word = '_' * len(word)

        while lives > 0 and '_' in guessed_word:
            letter = self.guess(guessed_word)
            if letter:
                self.guessed_letters.append(letter)
                if letter in word:
                    guessed_word = "".join(letter if word[i] == letter else guessed_word[i] for i in range(len(word)))
                else:
                    lives -= 1

        return lives

    def start_game(self, practice=True, verbose=True):
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice": practice})
        if response.get('status') == "approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print(f"Successfully started a new game! Game ID: {game_id}. # of tries remaining: {tries_remains}. Word: {word}.")
            while tries_remains > 0:
                guess_letter = self.guess(word)
                
                if guess_letter:
                    self.guessed_letters.append(guess_letter)
                    if verbose:
                        print(f"Guessing letter: {guess_letter}")
                    
                    try:
                        res = self.request("/guess_letter", {"request": "guess_letter", "game_id": game_id, "letter": guess_letter})
                    except HangmanAPIError:
                        print('HangmanAPIError exception caught on request.')
                        continue
                    except Exception as e:
                        print('Other exception caught on request.')
                        raise e
                    
                    if verbose:
                        print(f"Server response: {res}")
                    status = res.get('status')
                    tries_remains = res.get('tries_remains')
                    if status == "success":
                        if verbose:
                            print(f"Successfully finished game: {game_id}")
                        return True
                    elif status == "failed":
                        reason = res.get('reason', '# of tries exceeded!')
                        if verbose:
                            print(f"Failed game: {game_id}. Because of: {reason}")
                        return False
                    elif status == "ongoing":
                        word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status == "success"
    
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        if self.access_token:
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

### Weighted ngram, pre/suffix

In [380]:
import collections
import re
import requests
import string
import time


class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
        # Additional variables for n-gram analysis
        self.nseq_freq_list = []
        self.prefix_freq_list = []
        self.suffix_freq_list = []
        self.max_sequence_length = 5
        
        # Initialize frequency lists
        self.init_frequency_lists()

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']
        data = {link: 0 for link in links}

        for link in links:
            requests.get(link)
            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def build_dictionary(self, dictionary_file_location):
        with open(dictionary_file_location, "r") as text_file:
            full_dict = text_file.read().splitlines()
        return full_dict

    def init_frequency_lists(self):
        """Initialize n-gram, prefix, and suffix frequency lists."""
        for i in range(self.max_sequence_length):
            n_seq_freq, pre_freq, suf_freq = self.create_freq(self.full_dictionary, i + 1)
            self.nseq_freq_list.append(n_seq_freq)
            self.prefix_freq_list.append(pre_freq)
            self.suffix_freq_list.append(suf_freq)

    def create_freq(self, dictionary, n):
        """Create frequency dictionaries for n-grams, prefixes, and suffixes."""
        n_seq_freq = {}
        pre_freq = {}
        suf_freq = {}

        for word in dictionary:
            if len(word) < n:
                continue

            # N-grams
            for i in range(len(word) - n + 1):
                ngram = word[i:i + n]
                n_seq_freq[ngram] = n_seq_freq.get(ngram, 0) + 1

            # Prefixes
            prefix = word[:n]
            pre_freq[prefix] = pre_freq.get(prefix, 0) + 1

            # Suffixes
            suffix = word[-n:]
            suf_freq[suffix] = suf_freq.get(suffix, 0) + 1

        return n_seq_freq, pre_freq, suf_freq

    def getCount(self, temp, i, guessed_but_not_found_letters):
        """Helper function to count n-grams not including guessed letters."""
        count = 0
        for ngram, freq in sorted(temp.items(), key=lambda x: x[1], reverse=True):
            curr = list(ngram)
            for pos in range(i + 1):
                if curr[pos] in guessed_but_not_found_letters:
                    break
                count += freq
        return count

    def guess(self, word):
        """Enhanced guessing function using n-gram analysis."""
        clean_word = word[::2].replace("_", ".")
        guessed_but_not_found_letters = {letter for letter in self.guessed_letters if letter not in clean_word}
        len_word = len(clean_word)
        blanks_pct = clean_word.count('.') / len_word

        new_dictionary = []

        # Update current dictionary
        for dict_word in self.current_dictionary:
            if len(dict_word) == len_word and re.match(clean_word, dict_word):
                if not any(char in guessed_but_not_found_letters for char in dict_word):
                    new_dictionary.append(dict_word)

        self.current_dictionary = new_dictionary

        if blanks_pct >= 0.70 and self.current_dictionary:
            freq_table_init = collections.Counter("".join(self.current_dictionary)).most_common()
            for letter, _ in freq_table_init:
                if letter not in self.guessed_letters:
                    return letter
        else:
            freq_table_lat = collections.defaultdict(int)
            wt_arr = [1, 10, 100, 1000, 10000, 100000, 1000000]

            for i in range(self.max_sequence_length):
                temp = self.nseq_freq_list[i]
                tempCount = self.getCount(temp, i, guessed_but_not_found_letters)

                for ngram, freq in sorted(temp.items(), key=lambda x: x[1], reverse=True):
                    freq /= tempCount
                    uess_letters = [curr[pos] for pos, curr in enumerate(ngram) if pos < len(curr) and curr[pos] not in self.guessed_letters]
                    if guess_letters:
                        freq_table_lat[guess_letters[0]] += wt_arr[i] * freq

                # Similar logic for prefixes and suffixes

            for letter, _ in sorted(freq_table_lat.items(), key=lambda x: x[1], reverse=True):
                if letter not in self.guessed_letters:
                    return letter

        for letter, _ in self.full_dictionary_common_letter_sorted:
            if letter not in self.guessed_letters:
                return letter

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

### GPT 1

In [27]:
# import collections
# import re
# import random
# import time

# class TrieNode:
#     def __init__(self):
#         self.children = {}
#         self.is_end_of_word = False

# class Trie:
#     def __init__(self):
#         self.root = TrieNode()

#     def insert(self, word):
#         node = self.root
#         for letter in word:
#             if letter not in node.children:
#                 node.children[letter] = TrieNode()
#             node = node.children[letter]
#         node.is_end_of_word = True

#     def search(self, prefix):
#         node = self.root
#         for letter in prefix:
#             if letter not in node.children:
#                 return False
#             node = node.children[letter]
#         return True

# class HangmanGame:
#     def __init__(self, dictionary_file_location="words.txt"):
#         self.full_dictionary = self.build_dictionary(dictionary_file_location)
#         self.current_dictionary = self.full_dictionary
#         self.trie = Trie()
#         self.build_trie(self.full_dictionary)
#         self.popchars = ""
#         self.reinforcement_memory = {}
#         self.update_popchars()

#     def build_dictionary(self, dictionary_file_location):
#         with open(dictionary_file_location, "r") as text_file:
#             return text_file.read().splitlines()

#     def build_trie(self, words):
#         for word in words:
#             self.trie.insert(word)

#     def update_popchars(self):
#         letter_freq = collections.Counter("".join(self.current_dictionary))
#         most_common = letter_freq.most_common(5)  # Update to top 5
#         self.popchars = "".join([char for char, freq in most_common])

#     def dynamic_learning(self, word, guessed_letters, incorrect):
#         if word not in self.reinforcement_memory:
#             self.reinforcement_memory[word] = {"correct": 0, "incorrect": 0}
        
#         if incorrect:
#             self.reinforcement_memory[word]["incorrect"] += 1
#         else:
#             self.reinforcement_memory[word]["correct"] += 1

#         # Adjust strategy if a word has been incorrectly guessed multiple times
#         if self.reinforcement_memory[word]["incorrect"] > 3:
#             guessed_letters.append(random.choice([char for char in "abcdefghijklmnopqrstuvwxyz" if char not in guessed_letters]))

#     def build_ngram_model(self, word_list, n=3):
#         ngram_dict = collections.defaultdict(int)
#         for word in word_list:
#             for i in range(len(word) - n + 1):
#                 ngram = word[i:i + n]
#                 ngram_dict[ngram] += 1
#         return ngram_dict

#     def guess(self, word, guessed_letters):
#         clean_word = word.replace("_", ".")
#         len_word = len(clean_word)
#         new_dictionary = [dict_word for dict_word in self.current_dictionary if len(dict_word) == len_word and re.match(clean_word, dict_word)]
#         self.current_dictionary = new_dictionary

#         self.update_popchars()

#         # N-gram frequency-based guessing
#         ngram_model = self.build_ngram_model(self.current_dictionary)
#         sorted_ngram = sorted(ngram_model.items(), key=lambda x: x[1], reverse=True)

#         for ngram, _ in sorted_ngram:
#             for letter in ngram:
#                 if letter not in guessed_letters:
#                     guessed_letters.append(letter)
#                     return letter

#         # If no letter guessed, fallback to most common letters
#         sorted_letter_count = collections.Counter("".join(self.current_dictionary)).most_common()
#         for letter, _ in sorted_letter_count:
#             if letter not in guessed_letters:
#                 guessed_letters.append(letter)
#                 return letter

#         # Random guess if all else fails
#         return random.choice([char for char in "abcdefghijklmnopqrstuvwxyz" if char not in guessed_letters])

#     def simulate_hangman(self, word):
#         self.current_dictionary = self.full_dictionary
#         self.guessed_letters = []

#         max_lives = 6
#         lives = max_lives
#         guessed_word = '_' * len(word)

#         while lives > 0 and '_' in guessed_word:
#             letter = self.guess(guessed_word)
#             if letter:
#                 self.guessed_letters.append(letter)
#                 if letter in word:
#                     guessed_word = "".join(letter if word[i] == letter else guessed_word[i] for i in range(len(word)))
#                 else:
#                     lives -= 1

#         return lives

#     def start_game(self, practice=True, verbose=True):
#         self.guessed_letters = []
#         self.current_dictionary = self.full_dictionary
                         
#         response = self.request("/new_game", {"practice": practice})
#         if response.get('status') == "approved":
#             game_id = response.get('game_id')
#             word = response.get('word')
#             tries_remains = response.get('tries_remains')
#             if verbose:
#                 print(f"Successfully started a new game! Game ID: {game_id}. # of tries remaining: {tries_remains}. Word: {word}.")
#             while tries_remains > 0:
#                 guess_letter = self.guess(word)
                
#                 if guess_letter:
#                     self.guessed_letters.append(guess_letter)
#                     if verbose:
#                         print(f"Guessing letter: {guess_letter}")
                    
#                     try:
#                         res = self.request("/guess_letter", {"request": "guess_letter", "game_id": game_id, "letter": guess_letter})
#                     except HangmanAPIError:
#                         print('HangmanAPIError exception caught on request.')
#                         continue
#                     except Exception as e:
#                         print('Other exception caught on request.')
#                         raise e
                    
#                     if verbose:
#                         print(f"Server response: {res}")
#                     status = res.get('status')
#                     tries_remains = res.get('tries_remains')
#                     if status == "success":
#                         if verbose:
#                             print(f"Successfully finished game: {game_id}")
#                         return True
#                     elif status == "failed":
#                         reason = res.get('reason', '# of tries exceeded!')
#                         if verbose:
#                             print(f"Failed game: {game_id}. Because of: {reason}")
#                         return False
#                     elif status == "ongoing":
#                         word = res.get('word')
#         else:
#             if verbose:
#                 print("Failed to start a new game")
#         return status == "success"
    
#     def my_status(self):
#         return self.request("/my_status", {})
    
#     def request(self, path, args=None, post_args=None, method=None):
#         if args is None:
#             args = dict()
#         if post_args is not None:
#             method = "POST"

#         if self.access_token:
#             if post_args and "access_token" not in post_args:
#                 post_args["access_token"] = self.access_token
#             elif "access_token" not in args:
#                 args["access_token"] = self.access_token

#         time.sleep(0.2)

#         num_retry, time_sleep = 50, 2
#         for it in range(num_retry):
#             try:
#                 response = self.session.request(
#                     method or "GET",
#                     self.hangman_url + path,
#                     timeout=self.timeout,
#                     params=args,
#                     data=post_args,
#                     verify=False
#                 )
#                 break
#             except requests.HTTPError as e:
#                 response = json.loads(e.read())
#                 raise HangmanAPIError(response)
#             except requests.exceptions.SSLError as e:
#                 if it + 1 == num_retry:
#                     raise
#                 time.sleep(time_sleep)

#         headers = response.headers
#         if 'json' in headers['content-type']:
#             result = response.json()
#         elif "access_token" in parse_qs(response.text):
#             query_str = parse_qs(response.text)
#             if "access_token" in query_str:
#                 result = {"access_token": query_str["access_token"][0]}
#                 if "expires" in query_str:
#                     result["expires"] = query_str["expires"][0]
#             else:
#                 raise HangmanAPIError(response.json())
#         else:
#             raise HangmanAPIError('Maintype was not text, or querystring')

#         if result and isinstance(result, dict) and result.get("error"):
#             raise HangmanAPIError(result)
#         return result

In [ ]:
# class HangmanAPI(object):
#     def __init__(self, access_token=None, session=None, timeout=None):
#         self.hangman_url = self.determine_hangman_url()
#         self.access_token = access_token
#         self.session = session or requests.Session()
#         self.timeout = timeout
#         self.guessed_letters = []
        
#         full_dictionary_location = "words.txt"
#         self.full_dictionary = self.build_dictionary(full_dictionary_location)        
#         self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
#         self.current_dictionary = []
        
#     @staticmethod
#     def determine_hangman_url():
#         links = ['https://trexsim.com', 'https://sg.trexsim.com']

#         data = {link: 0 for link in links}

#         for link in links:

#             requests.get(link)

#             for i in range(10):
#                 s = time.time()
#                 requests.get(link)
#                 data[link] = time.time() - s

#         link = sorted(data.items(), key=lambda x: x[1])[0][0]
#         link += '/trexsim/hangman'
#         return link

#     def guess(self, word): # word input example: "_ p p _ e "
#         ###############################################
#         # Replace with your own "guess" function here #
#         ###############################################

#         # clean the word so that we strip away the space characters
#         # replace "_" with "." as "." indicates any character in regular expressions
#         clean_word = word[::2].replace("_",".")
        
#         # find length of passed word
#         len_word = len(clean_word)
        
#         # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
#         current_dictionary = self.current_dictionary
#         new_dictionary = []
        
#         # iterate through all of the words in the old plausible dictionary
#         for dict_word in current_dictionary:
#             # continue if the word is not of the appropriate length
#             if len(dict_word) != len_word:
#                 continue
                
#             # if dictionary word is a possible match then add it to the current dictionary
#             if re.match(clean_word,dict_word):
#                 new_dictionary.append(dict_word)
        
#         # overwrite old possible words dictionary with updated version
#         self.current_dictionary = new_dictionary
        
        
#         # count occurrence of all characters in possible word matches
#         full_dict_string = "".join(new_dictionary)
        
#         c = collections.Counter(full_dict_string)
#         sorted_letter_count = c.most_common()                   
        
#         guess_letter = '!'
        
#         # return most frequently occurring letter in all possible words that hasn't been guessed yet
#         for letter,instance_count in sorted_letter_count:
#             if letter not in self.guessed_letters:
#                 guess_letter = letter
#                 break
            
#         # if no word matches in training dictionary, default back to ordering of full dictionary
#         if guess_letter == '!':
#             sorted_letter_count = self.full_dictionary_common_letter_sorted
#             for letter,instance_count in sorted_letter_count:
#                 if letter not in self.guessed_letters:
#                     guess_letter = letter
#                     break            
        
#         return guess_letter

#     ##########################################################
#     # You'll likely not need to modify any of the code below #
#     ##########################################################
    
#     def build_dictionary(self, dictionary_file_location):
#         text_file = open(dictionary_file_location,"r")
#         full_dictionary = text_file.read().splitlines()
#         text_file.close()
#         return full_dictionary
                
#     def start_game(self, practice=True, verbose=True):
#         # reset guessed letters to empty set and current plausible dictionary to the full dictionary
#         self.guessed_letters = []
#         self.current_dictionary = self.full_dictionary
                         
#         response = self.request("/new_game", {"practice":practice})
#         if response.get('status')=="approved":
#             game_id = response.get('game_id')
#             word = response.get('word')
#             tries_remains = response.get('tries_remains')
#             if verbose:
#                 print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
#             while tries_remains>0:
#                 # get guessed letter from user code
#                 guess_letter = self.guess(word)
                    
#                 # append guessed letter to guessed letters field in hangman object
#                 self.guessed_letters.append(guess_letter)
#                 if verbose:
#                     print("Guessing letter: {0}".format(guess_letter))
                    
#                 try:    
#                     res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
#                 except HangmanAPIError:
#                     print('HangmanAPIError exception caught on request.')
#                     continue
#                 except Exception as e:
#                     print('Other exception caught on request.')
#                     raise e
               
#                 if verbose:
#                     print("Sever response: {0}".format(res))
#                 status = res.get('status')
#                 tries_remains = res.get('tries_remains')
#                 if status=="success":
#                     if verbose:
#                         print("Successfully finished game: {0}".format(game_id))
#                     return True
#                 elif status=="failed":
#                     reason = res.get('reason', '# of tries exceeded!')
#                     if verbose:
#                         print("Failed game: {0}. Because of: {1}".format(game_id, reason))
#                     return False
#                 elif status=="ongoing":
#                     word = res.get('word')
#         else:
#             if verbose:
#                 print("Failed to start a new game")
#         return status=="success"
        
#     def my_status(self):
#         return self.request("/my_status", {})
    
#     def request(
#             self, path, args=None, post_args=None, method=None):
#         if args is None:
#             args = dict()
#         if post_args is not None:
#             method = "POST"

#         # Add `access_token` to post_args or args if it has not already been
#         # included.
#         if self.access_token:
#             # If post_args exists, we assume that args either does not exists
#             # or it does not need `access_token`.
#             if post_args and "access_token" not in post_args:
#                 post_args["access_token"] = self.access_token
#             elif "access_token" not in args:
#                 args["access_token"] = self.access_token

#         time.sleep(0.2)

#         num_retry, time_sleep = 50, 2
#         for it in range(num_retry):
#             try:
#                 response = self.session.request(
#                     method or "GET",
#                     self.hangman_url + path,
#                     timeout=self.timeout,
#                     params=args,
#                     data=post_args,
#                     verify=False
#                 )
#                 break
#             except requests.HTTPError as e:
#                 response = json.loads(e.read())
#                 raise HangmanAPIError(response)
#             except requests.exceptions.SSLError as e:
#                 if it + 1 == num_retry:
#                     raise
#                 time.sleep(time_sleep)

#         headers = response.headers
#         if 'json' in headers['content-type']:
#             result = response.json()
#         elif "access_token" in parse_qs(response.text):
#             query_str = parse_qs(response.text)
#             if "access_token" in query_str:
#                 result = {"access_token": query_str["access_token"][0]}
#                 if "expires" in query_str:
#                     result["expires"] = query_str["expires"][0]
#             else:
#                 raise HangmanAPIError(response.json())
#         else:
#             raise HangmanAPIError('Maintype was not text, or querystring')

#         if result and isinstance(result, dict) and result.get("error"):
#             raise HangmanAPIError(result)
#         return result
    
# class HangmanAPIError(Exception):
#     def __init__(self, result):
#         self.result = result
#         self.code = None
#         try:
#             self.type = result["error_code"]
#         except (KeyError, TypeError):
#             self.type = ""

#         try:
#             self.message = result["error_description"]
#         except (KeyError, TypeError):
#             try:
#                 self.message = result["error"]["message"]
#                 self.code = result["error"].get("code")
#                 if not self.type:
#                     self.type = result["error"].get("type", "")
#             except (KeyError, TypeError):
#                 try:
#                     self.message = result["error_msg"]
#                 except (KeyError, TypeError):
#                     self.message = result

#         Exception.__init__(self, self.message)

In [45]:
# class HangmanAPI(object):
#     def __init__(self, access_token=None, session=None, timeout=None):
#         self.hangman_url = self.determine_hangman_url()
#         self.access_token = access_token
#         self.session = session or requests.Session()
#         self.timeout = timeout
#         self.guessed_letters = []
        
#         full_dictionary_location = "words.txt"
#         self.full_dictionary = self.build_dictionary(full_dictionary_location)        
#         self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
#         self.current_dictionary = []
#         self.trie = self.build_trie(self.full_dictionary)
        
#     @staticmethod
#     def determine_hangman_url():
#         links = ['https://trexsim.com', 'https://sg.trexsim.com']

#         data = {link: 0 for link in links}

#         for link in links:

#             requests.get(link)

#             for i in range(10):
#                 s = time.time()
#                 requests.get(link)
#                 data[link] = time.time() - s

#         link = sorted(data.items(), key=lambda x: x[1])[0][0]
#         link += '/trexsim/hangman'
#         return link

#     def build_dictionary(self, dictionary_file_location):
#         text_file = open(dictionary_file_location, "r")
#         full_dictionary = text_file.read().splitlines()
#         text_file.close()
#         return full_dictionary

#     def build_trie(self, words):
#         """Build a Trie for fast word lookup (Optional)"""
#         class TrieNode:
#             def __init__(self):
#                 self.children = {}
#                 self.is_word = False

#         root = TrieNode()
#         for word in words:
#             current = root
#             for letter in word:
#                 if letter not in current.children:
#                     current.children[letter] = TrieNode()
#                 current = current.children[letter]
#             current.is_word = True
#         return root

#     def update_possible_words(self, pattern, guessed_letters, current_possible_words):
#         """Update the possible words based on the current pattern and guessed letters"""
#         pattern = pattern.lower()
#         word_length = len(pattern)

#         known_letters = set(pattern.replace('_', ''))
#         incorrect_letters = set(guessed_letters) - known_letters
#         new_possible_words = []

#         for word in current_possible_words:
#             match = True
#             for p_char, w_char in zip(pattern, word):
#                 if p_char == '_':
#                     if w_char in known_letters or w_char in incorrect_letters:
#                         match = False
#                         break
#                 else:
#                     if w_char != p_char:
#                         match = False
#                         break
#             if match and not any(letter in incorrect_letters for letter in word):
#                 new_possible_words.append(word)

#         return new_possible_words

#     def calculate_pattern_entropies(self, current_possible_words, guessed_letters):
#         """Calculate the entropy for possible letters"""
#         total_possible_words = len(current_possible_words)
#         if total_possible_words == 0:
#             return {}

#         letter_entropies = {}
#         possible_letters = set('abcdefghijklmnopqrstuvwxyz') - set(guessed_letters)

#         for letter in possible_letters:
#             pattern_counts = collections.Counter()
#             for word in current_possible_words:
#                 pattern = ['_' if c != letter else letter for c in word]
#                 pattern_key = ''.join(pattern)
#                 pattern_counts[pattern_key] += 1

#             entropy = 0
#             for count in pattern_counts.values():
#                 p = count / total_possible_words
#                 entropy -= p * math.log2(p)
#             letter_entropies[letter] = entropy

#         return letter_entropies

#     def select_next_letter(self, current_possible_words, guessed_letters):
#         """Select the next best letter based on entropy calculations"""
#         letter_entropies = self.calculate_pattern_entropies(current_possible_words, guessed_letters)
#         if not letter_entropies:
#             return None
#         next_letter = max(letter_entropies, key=letter_entropies.get)
#         return next_letter

#     def guess(self, word): 
#         """Guess a letter based on the current state of the game"""
#         pattern = word[::2].replace("_", ".")
#         len_word = len(pattern)

#         self.current_dictionary = self.update_possible_words(pattern, self.guessed_letters, self.current_dictionary)

#         next_letter = self.select_next_letter(self.current_dictionary, self.guessed_letters)
#         if next_letter:
#             return next_letter

#         # Fallback if no letter is found by entropy method
#         sorted_letter_count = self.full_dictionary_common_letter_sorted
#         for letter, _ in sorted_letter_count:
#             if letter not in self.guessed_letters:
#                 return letter

#         return '!'  # If no suitable guess found (very rare)

#     def dynamic_learning(self, word, incorrect=False, reinforcement_memory=None):
#         """Simulate dynamic learning for the reinforcement memory (optional)"""
#         if incorrect:
#             # Store dynamic memory or learning based on incorrect guesses
#             pass

#     def start_game(self, practice=True, verbose=True):
#         """Start a new Hangman game and manage the guessing loop"""
#         self.guessed_letters = []
#         self.current_dictionary = self.full_dictionary

#         response = self.request("/new_game", {"practice": practice})
#         if response.get('status') == "approved":
#             game_id = response.get('game_id')
#             word = response.get('word')
#             tries_remains = response.get('tries_remains')
#             if verbose:
#                 print(f"Successfully started a new game! Game ID: {game_id}. Tries remaining: {tries_remains}. Word: {word}.")
            
#             while tries_remains > 0:
#                 guess_letter = self.guess(word)
#                 self.guessed_letters.append(guess_letter)
                
#                 if verbose:
#                     print(f"Guessing letter: {guess_letter}")
                
#                 res = self.request("/guess_letter", {"request": "guess_letter", "game_id": game_id, "letter": guess_letter})
#                 status = res.get('status')
#                 tries_remains = res.get('tries_remains')

#                 if status == "success":
#                     if verbose:
#                         print(f"Successfully finished game: {game_id}")
#                     return True
#                 elif status == "failed":
#                     if verbose:
#                         print(f"Failed game: {game_id}")
#                     return False
#                 elif status == "ongoing":
#                     word = res.get('word')

#         if verbose:
#             print("Failed to start a new game")
#         return False

        
#     def my_status(self):
#         return self.request("/my_status", {})
    
#     def request(
#             self, path, args=None, post_args=None, method=None):
#         if args is None:
#             args = dict()
#         if post_args is not None:
#             method = "POST"

#         # Add `access_token` to post_args or args if it has not already been
#         # included.
#         if self.access_token:
#             # If post_args exists, we assume that args either does not exists
#             # or it does not need `access_token`.
#             if post_args and "access_token" not in post_args:
#                 post_args["access_token"] = self.access_token
#             elif "access_token" not in args:
#                 args["access_token"] = self.access_token

#         time.sleep(0.2)

#         num_retry, time_sleep = 50, 2
#         for it in range(num_retry):
#             try:
#                 response = self.session.request(
#                     method or "GET",
#                     self.hangman_url + path,
#                     timeout=self.timeout,
#                     params=args,
#                     data=post_args,
#                     verify=False
#                 )
#                 break
#             except requests.HTTPError as e:
#                 response = json.loads(e.read())
#                 raise HangmanAPIError(response)
#             except requests.exceptions.SSLError as e:
#                 if it + 1 == num_retry:
#                     raise
#                 time.sleep(time_sleep)

#         headers = response.headers
#         if 'json' in headers['content-type']:
#             result = response.json()
#         elif "access_token" in parse_qs(response.text):
#             query_str = parse_qs(response.text)
#             if "access_token" in query_str:
#                 result = {"access_token": query_str["access_token"][0]}
#                 if "expires" in query_str:
#                     result["expires"] = query_str["expires"][0]
#             else:
#                 raise HangmanAPIError(response.json())
#         else:
#             raise HangmanAPIError('Maintype was not text, or querystring')

#         if result and isinstance(result, dict) and result.get("error"):
#             raise HangmanAPIError(result)
#         return result
    
# class HangmanAPIError(Exception):
#     def __init__(self, result):
#         self.result = result
#         self.code = None
#         try:
#             self.type = result["error_code"]
#         except (KeyError, TypeError):
#             self.type = ""

#         try:
#             self.message = result["error_description"]
#         except (KeyError, TypeError):
#             try:
#                 self.message = result["error"]["message"]
#                 self.code = result["error"].get("code")
#                 if not self.type:
#                     self.type = result["error"].get("type", "")
#             except (KeyError, TypeError):
#                 try:
#                     self.message = result["error_msg"]
#                 except (KeyError, TypeError):
#                     self.message = result

#         Exception.__init__(self, self.message)

In [365]:
import re
import collections
import time
import requests
from requests.exceptions import HTTPError

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "words.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.n_gram_model = self.build_n_gram_model(self.full_dictionary)
        self.current_dictionary = self.full_dictionary

    def build_dictionary(self, dictionary_file_location):
        with open(dictionary_file_location, "r") as text_file:
            full_dictionary = text_file.read().splitlines()
        return full_dictionary

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:
            requests.get(link)
            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def build_n_gram_model(self, dictionary):
        # Create n-gram frequencies for n = 1 to 4
        n_grams = {}
        for word in dictionary:
            for i in range(len(word)):
                for n in range(1, 5):  # Create n-grams for n = 1 to 4
                    gram = word[i:i+n]
                    if len(gram) == n:
                        n_grams[gram] = n_grams.get(gram, 0) + 1
        return n_grams
    
    def update_dictionary(self, current_pattern):
        # Create a regex pattern to match the current word state
        regex_pattern = ''.join(['.' if char == '_' else char for char in current_pattern])
        self.current_dictionary = [word for word in self.current_dictionary if re.fullmatch(regex_pattern, word)]


    def guess(self, word):
        # Clean the word by removing extra spaces, and replace underscores with dots for regex pattern
        clean_word = word[::2].replace("_", ".")
        len_word = len(clean_word)

        # Update the dictionary by filtering words that match the current pattern
        self.current_dictionary = [w for w in self.current_dictionary if len(w) == len_word and re.match(clean_word, w)]

        # After filtering, if there's only one word left, focus on that word
        if len(self.current_dictionary) == 1:
            target_word = self.current_dictionary[0]
            for letter in target_word:
                if letter not in self.guessed_letters:
                    self.guessed_letters.append(letter)
                    return letter

        # Calculate letter frequency based on the remaining words in the dictionary
        letter_frequencies = self.calculate_letter_frequencies()

        # Filter out already guessed letters
        for letter in self.guessed_letters:
            if letter in letter_frequencies:
                del letter_frequencies[letter]

        # Guess the letter with the highest frequency in the remaining dictionary
        best_guess = max(letter_frequencies, key=letter_frequencies.get, default=None)

        if best_guess:
            self.guessed_letters.append(best_guess)
            return best_guess
        else:
            # If no more guesses are available, fallback to most common unguessed letters
            return self.guess_most_common()

    def calculate_letter_frequencies(self):
        frequencies = {}
        for word in self.current_dictionary:
            for letter in word:
                if letter not in self.guessed_letters:
                    frequencies[letter] = frequencies.get(letter, 0) + 1
        return frequencies

    def get_n_gram_scores(self, clean_word):
        letter_scores = {}

        # Focus on the most likely n-grams given the current word pattern
        for i in range(len(clean_word)):
            if clean_word[i] == ".":  # Only consider unguessed spots
                for n in range(1, 5):  # Weighted n-gram approach
                    n_gram = clean_word[max(0, i-n+1):i+1]
                    if n_gram in self.n_gram_model:
                        # Predict the next character in the word
                        next_letter_index = i + 1
                        if next_letter_index < len(clean_word):
                            possible_next = clean_word[next_letter_index]
                            for word in self.current_dictionary:
                                if word[i] not in self.guessed_letters:
                                    letter_scores[word[i]] = letter_scores.get(word[i], 0) + self.n_gram_model[n_gram] * n

        return letter_scores


    def matches_pattern(self, n_gram, pattern, index):
        # Ensure the n-gram fits within the current word pattern (for given index)
        for i in range(len(n_gram)):
            if pattern[index + i] != "." and pattern[index + i] != n_gram[i]:
                return False
        return True

    def guess_most_common(self):
        # Create a single string of all words in the full dictionary
        full_dict_string = "".join(self.full_dictionary)

        # Get the count of each letter in the full dictionary
        letter_frequencies = collections.Counter(full_dict_string)

        # Sort letters by frequency, highest first
        sorted_letter_frequencies = letter_frequencies.most_common()

        # Filter the letters to return the most frequent, non-guessed letter
        for letter, count in sorted_letter_frequencies:
            if letter not in self.guessed_letters and letter.isalpha():
                return letter


    def start_game(self, practice=True, verbose=True):
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice": practice})
        if response.get('status') == "approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print(f"Successfully started a new game! Game ID: {game_id}. # of tries remaining: {tries_remains}. Word: {word}.")
            while tries_remains > 0:
                guess_letter = self.guess(word)

                if guess_letter:
                    self.guessed_letters.append(guess_letter)
                    if verbose:
                        print(f"Guessing letter: {guess_letter}")

                    try:
                        res = self.request("/guess_letter", {"request": "guess_letter", "game_id": game_id, "letter": guess_letter})
                    except HangmanAPIError:
                        print('HangmanAPIError exception caught on request.')
                        continue
                    except Exception as e:
                        print('Other exception caught on request.')
                        raise e

                    if verbose:
                        print(f"Server response: {res}")
                    status = res.get('status')
                    tries_remains = res.get('tries_remains')
                    if status == "success":
                        if verbose:
                            print(f"Successfully finished game: {game_id}")
                        return True
                    elif status == "failed":
                        reason = res.get('reason', '# of tries exceeded!')
                        if verbose:
                            print(f"Failed game: {game_id}. Because of: {reason}")
                        return False
                    elif status == "ongoing":
                        word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return False

    def my_status(self):
        return self.request("/my_status", {})

    def request(self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        if self.access_token:
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result


In [376]:
import re
import collections
import time
import requests
from requests.exceptions import HTTPError

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "words.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.current_dictionary = self.full_dictionary

    def build_dictionary(self, dictionary_file_location):
        with open(dictionary_file_location, "r") as text_file:
            full_dictionary = text_file.read().splitlines()
        return full_dictionary

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:
            requests.get(link)
            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def update_dictionary(self, current_pattern):
        # Create a regex pattern to match the current word state
        regex_pattern = ''.join(['.' if char == '_' else char for char in current_pattern])
        self.current_dictionary = [word for word in self.current_dictionary if re.fullmatch(regex_pattern, word)]

    def guess(self, word):
        # Clean the word by removing extra spaces and replacing underscores with dots for regex pattern
        clean_word = word[::2].replace("_", ".")
        len_word = len(clean_word)

        # Update the dictionary by filtering words that match the current pattern
        self.current_dictionary = [w for w in self.current_dictionary if len(w) == len_word and re.match(clean_word, w)]

        # After filtering, if there's only one word left, guess the letters in that word
        if len(self.current_dictionary) == 1:
            target_word = self.current_dictionary[0]
            for letter in target_word:
                if letter not in self.guessed_letters:
                    self.guessed_letters.append(letter)
                    return letter

        # Calculate letter frequency based on the remaining words in the dictionary
        letter_frequencies = self.calculate_letter_frequencies()

        # Filter out already guessed letters
        for letter in self.guessed_letters:
            if letter in letter_frequencies:
                del letter_frequencies[letter]

        # Guess the letter with the highest frequency in the remaining dictionary
        best_guess = max(letter_frequencies, key=letter_frequencies.get, default=None)

        if best_guess:
            self.guessed_letters.append(best_guess)
            return best_guess
        else:
            # If no more guesses are available, fallback to most common unguessed letters
            return self.guess_most_common()

    def calculate_letter_frequencies(self):
        frequencies = {}
        for word in self.current_dictionary:
            for letter in word:
                if letter not in self.guessed_letters:
                    frequencies[letter] = frequencies.get(letter, 0) + 1

        # Improve the strategy by considering n-grams
        letter_scores = self.get_n_gram_scores(frequencies)

        # Return letter scores for the best guess
        return letter_scores

    def get_n_gram_scores(self, frequencies):
        letter_scores = collections.Counter()

        for word in self.current_dictionary:
            for i in range(len(word)):
                if word[i] not in self.guessed_letters:
                    # Increase the score based on the frequency of the current letter
                    letter_scores[word[i]] += frequencies[word[i]]

        return letter_scores

    def guess_most_common(self):
        # Create a single string of all words in the full dictionary
        full_dict_string = "".join(self.full_dictionary)

        # Get the count of each letter in the full dictionary
        letter_frequencies = collections.Counter(full_dict_string)

        # Sort letters by frequency, highest first
        sorted_letter_frequencies = letter_frequencies.most_common()

        # Filter the letters to return the most frequent, non-guessed letter
        for letter, count in sorted_letter_frequencies:
            if letter not in self.guessed_letters and letter.isalpha():
                return letter

    def start_game(self, practice=True, verbose=True):
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice": practice})
        if response.get('status') == "approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print(f"Successfully started a new game! Game ID: {game_id}. # of tries remaining: {tries_remains}. Word: {word}.")
            while tries_remains > 0:
                guess_letter = self.guess(word)

                if guess_letter:
                    self.guessed_letters.append(guess_letter)
                    if verbose:
                        print(f"Guessing letter: {guess_letter}")

                    try:
                        res = self.request("/guess_letter", {"request": "guess_letter", "game_id": game_id, "letter": guess_letter})
                    except HangmanAPIError:
                        print('HangmanAPIError exception caught on request.')
                        continue
                    except Exception as e:
                        print('Other exception caught on request.')
                        raise e

                    if verbose:
                        print(f"Server response: {res}")
                    status = res.get('status')
                    tries_remains = res.get('tries_remains')
                    if status == "success":
                        if verbose:
                            print(f"Successfully finished game: {game_id}")
                        return True
                    elif status == "failed":
                        reason = res.get('reason', '# of tries exceeded!')
                        if verbose:
                            print(f"Failed game: {game_id}. Because of: {reason}")
                        return False
                    elif status == "ongoing":
                        word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return False

    def my_status(self):
        return self.request("/my_status", {})

    def request(self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        if self.access_token:
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result


# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [381]:
api = HangmanAPI(access_token="f962ca36a254d5d4bacd01d4e4497b", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [382]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

Successfully start a new game! Game ID: df259d03c890. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': 'df259d03c890', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ _ e '}
Guessing letter: i
Sever response: {'game_id': 'df259d03c890', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ i _ _ _ _ _ e '}
Guessing letter: a
Sever response: {'game_id': 'df259d03c890', 'status': 'ongoing', 'tries_remains': 6, 'word': 'a _ _ i _ _ a _ _ e '}


NameError: name 'guess_letters' is not defined

In [379]:
import time
for i in range(100):
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))
    time.sleep(3)

Successfully started a new game! Game ID: c731a6d1ab5e. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': 'c731a6d1ab5e', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ e _ _ _ e _ _ '}
Guessing letter: s
Server response: {'game_id': 'c731a6d1ab5e', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ e _ _ _ e s s '}
Guessing letter: l
Server response: {'game_id': 'c731a6d1ab5e', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ e _ _ _ e s s '}
Guessing letter: n
Server response: {'game_id': 'c731a6d1ab5e', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ e _ _ _ e s s '}
Guessing letter: r
Server response: {'game_id': 'c731a6d1ab5e', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ e r _ r e s s '}
Guessing letter: i
Server response: {'game_id': 'c731a6d1ab5e', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ e r _ r e s s '}
Guessing letter: a
Server response: {'game_id': 'c731a6d1ab5e', 'status': 'ongoing'

Guessing letter: p
Server response: {'game_id': '9f65fb3a9173', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ l o o d t i n c t u r e d '}
Guessing letter: m
Server response: {'game_id': '9f65fb3a9173', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ l o o d t i n c t u r e d '}
Guessing letter: h
Server response: {'game_id': '9f65fb3a9173', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ l o o d t i n c t u r e d '}
Guessing letter: g
Server response: {'game_id': '9f65fb3a9173', 'status': 'failed', 'tries_remains': 0, 'word': '_ l o o d t i n c t u r e d '}
Failed game: 9f65fb3a9173. Because of: # of tries exceeded!
run 112 practice games out of an allotted 100,000. practice success rate so far = 0.214
Successfully started a new game! Game ID: 317ca4e3c4e9. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '317ca4e3c4e9', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ e _ _ _ _ _ _ e _ _ _ '}
Guessing letter: r


Server response: {'game_id': '5cb19c1088d7', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ a l l _ _ _ _ _ e '}
Guessing letter: t
Server response: {'game_id': '5cb19c1088d7', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ a l l _ _ _ _ _ e '}
Guessing letter: w
Server response: {'game_id': '5cb19c1088d7', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ a l l _ _ _ _ _ e '}
Guessing letter: o
Server response: {'game_id': '5cb19c1088d7', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ a l l o _ _ o _ e '}
Guessing letter: n
Server response: {'game_id': '5cb19c1088d7', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ a l l o _ _ o _ e '}
Guessing letter: r
Server response: {'game_id': '5cb19c1088d7', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ a l l o _ _ o _ e '}
Guessing letter: s
Server response: {'game_id': '5cb19c1088d7', 'status': 'failed', 'tries_remains': 0, 'word': '_ a l l o _ _ o _ e '}
Failed game: 5cb19c1088d7. Because of: # of tries exceeded!
r

Server response: {'game_id': 'df8362e059b2', 'status': 'failed', 'tries_remains': 0, 'word': '_ o b s t e r s '}
Failed game: df8362e059b2. Because of: # of tries exceeded!
run 120 practice games out of an allotted 100,000. practice success rate so far = 0.208
Successfully started a new game! Game ID: 3a1bc7a41b2e. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '3a1bc7a41b2e', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ e _ _ e _ '}
Guessing letter: s
Server response: {'game_id': '3a1bc7a41b2e', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ e _ _ e _ '}
Guessing letter: d
Server response: {'game_id': '3a1bc7a41b2e', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ d _ _ _ e _ _ e d '}
Guessing letter: l
Server response: {'game_id': '3a1bc7a41b2e', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ l d _ _ _ e _ _ e d '}
Guessing letter: i
Server response: {'game_id': '3a1

Server response: {'game_id': '68bc9cb8ba1b', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ e _ a t o l o g i c a l '}
Guessing letter: d
Server response: {'game_id': '68bc9cb8ba1b', 'status': 'failed', 'tries_remains': 0, 'word': '_ e _ a t o l o g i c a l '}
Failed game: 68bc9cb8ba1b. Because of: # of tries exceeded!
run 124 practice games out of an allotted 100,000. practice success rate so far = 0.218
Successfully started a new game! Game ID: 768bf72a09fe. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '768bf72a09fe', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ e _ '}
Guessing letter: r
Server response: {'game_id': '768bf72a09fe', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ e r '}
Guessing letter: i
Server response: {'game_id': '768bf72a09fe', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ i _ _ i e r '}
Guessing letter: s
Server response: {'game_id': '768bf72a09fe', 'status': 

...

Server response: {'game_id': 'e91fb29ef902', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ e _ _ '}
Guessing letter: r
Server response: {'game_id': 'e91fb29ef902', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ r e _ _ '}
Guessing letter: t
Server response: {'game_id': 'e91fb29ef902', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ r e _ t '}
Guessing letter: n
Server response: {'game_id': 'e91fb29ef902', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ r e _ t '}
Guessing letter: b
Server response: {'game_id': 'e91fb29ef902', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ r e _ t '}
Guessing letter: p
Server response: {'game_id': 'e91fb29ef902', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ r e _ t '}
Guessing letter: f
Server response: {'game_id': 'e91fb29ef902', 'status': 'failed', 'tries_remains': 0, 'word': '_ r e _ t '}
Failed game: e91fb29ef902. Because of: # of tries exceeded!
run 184 practice games out of an allotted 100,000. practice success rat

run 188 practice games out of an allotted 100,000. practice success rate so far = 0.202
Successfully started a new game! Game ID: a41e21bb7aa3. # of tries remaining: 6. Word: _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': 'a41e21bb7aa3', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ '}
Guessing letter: a
Server response: {'game_id': 'a41e21bb7aa3', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ a _ _ _ _ _ '}
Guessing letter: r
Server response: {'game_id': 'a41e21bb7aa3', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ a _ _ _ _ _ '}
Guessing letter: n
Server response: {'game_id': 'a41e21bb7aa3', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ a _ _ _ _ _ '}
Guessing letter: s
Server response: {'game_id': 'a41e21bb7aa3', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ a _ _ _ _ _ '}
Guessing letter: l
Server response: {'game_id': 'a41e21bb7aa3', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ a _ _ _ _ _ '}
Guessing letter: i
Serv

run 193 practice games out of an allotted 100,000. practice success rate so far = 0.202
Successfully started a new game! Game ID: d492854fc266. # of tries remaining: 6. Word: _ _ _ _ _ .
Guessing letter: a
Server response: {'game_id': 'd492854fc266', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ a _ _ a '}
Guessing letter: n
Server response: {'game_id': 'd492854fc266', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ a _ n a '}
Guessing letter: d
Server response: {'game_id': 'd492854fc266', 'status': 'ongoing', 'tries_remains': 6, 'word': 'd a _ n a '}
Guessing letter: g
Server response: {'game_id': 'd492854fc266', 'status': 'ongoing', 'tries_remains': 5, 'word': 'd a _ n a '}
Guessing letter: h
Server response: {'game_id': 'd492854fc266', 'status': 'ongoing', 'tries_remains': 4, 'word': 'd a _ n a '}
Guessing letter: w
Server response: {'game_id': 'd492854fc266', 'status': 'ongoing', 'tries_remains': 3, 'word': 'd a _ n a '}
Guessing letter: e
Server response: {'game_id': 'd4

Server response: {'game_id': '0ea4c0724ebb', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ e '}
Guessing letter: a
Server response: {'game_id': '0ea4c0724ebb', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ a _ _ _ _ _ _ e '}
Guessing letter: i
Server response: {'game_id': '0ea4c0724ebb', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ a _ _ i _ _ _ e '}
Guessing letter: t
Server response: {'game_id': '0ea4c0724ebb', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ a _ _ i _ _ _ e '}
Guessing letter: l
Server response: {'game_id': '0ea4c0724ebb', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ a _ l i _ _ l e '}
Guessing letter: g
Server response: {'game_id': '0ea4c0724ebb', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ a _ l i _ _ l e '}
Guessing letter: c
Server response: {'game_id': '0ea4c0724ebb', 'status': 'ongoing', 'tries_remains': 4, 'word': 'c a _ l i c _ l e '}
Guessing letter: n
Server response: {'game_id': '0ea4c0724ebb', 'status': 

Server response: {'game_id': '39ebd39bbfed', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ n _ _ a r r e _ _ i n g '}
Guessing letter: o
Server response: {'game_id': '39ebd39bbfed', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ n _ _ a r r e _ _ i n g '}
Guessing letter: s
Server response: {'game_id': '39ebd39bbfed', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ n _ _ a r r e _ _ i n g '}
Guessing letter: t
Server response: {'game_id': '39ebd39bbfed', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ n _ _ a r r e _ _ i n g '}
Guessing letter: l
Server response: {'game_id': '39ebd39bbfed', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ n _ _ a r r e l l i n g '}
Guessing letter: c
Server response: {'game_id': '39ebd39bbfed', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ n _ _ a r r e l l i n g '}
Guessing letter: u
Server response: {'game_id': '39ebd39bbfed', 'status': 'ongoing', 'tries_remains': 2, 'word': 'u n _ u a r r e l l i n g '}
Guessing letter: d

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)